In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
from heapq import heappush, heappop
import time
import math
import tkinter as tk
from tkinter import ttk, filedialog, messagebox


# creating the initial maze class
class Maze:
    def __init__(self, filepath):
        with open(filepath) as maze_text:
            contents = maze_text.read().rstrip() 
            maze_1 = contents.split('\n')
            maze_start = []
            for row in maze_1:
                row_new = row.rstrip().split(',')
                maze_start.append(row_new)
        
        self.height = len(maze_start)
        self.width = max(len(line) for line in maze_start)

        if contents.count("S") != 1:
            raise Exception("maze must have exactly one Soko!")

        self.walls = []
        self.dropzones = []
        self.boxes = []
        self.doors = []
        self.keys = []

        for i in range(self.height):
            row = []
            for j in range(self.width):
                try:
                    if maze_start[i][j] == "S":
                        self.soko = (i, j)
                        row.append(False)
                    elif maze_start[i][j].startswith("Z-"):
                        label = maze_start[i][j].split("-")[1]
                        self.dropzones.append((i, j, label))
                        row.append(False)
                    elif maze_start[i][j].startswith("B-"):
                        label = maze_start[i][j].split("-")[1]
                        self.boxes.append((i, j, label))
                        row.append(False)
                    elif maze_start[i][j].startswith("K-"):
                        label = maze_start[i][j].split("-")[1]
                        self.keys.append((i, j, label))
                        row.append(False)
                    elif maze_start[i][j].startswith("D-"):
                        label = maze_start[i][j].split("-")[1]
                        self.doors.append((i, j, label))
                        row.append(False)
                    elif maze_start[i][j] == " ":
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(False)
            self.walls.append(row)

        # Create lookup dictionaries
        self.door_map = {(r, c): label for r, c, label in self.doors}
        self.key_map = {(r, c): label for r, c, label in self.keys}
        self.box_map = {(r, c): label for r, c, label in self.boxes}
        self.dropzone_map = {label: (r, c) for r, c, label in self.dropzones}

    # validating the neighbours
    def valid_neighbours(self, robot):
        row, col = robot
        candidates = [
            ("up", (row - 1, col)),
            ("down", (row + 1, col)),
            ("left", (row, col - 1)),
            ("right", (row, col + 1))
        ]

        neighbours = []
        for action, (r, c) in candidates:
            if 0 <= r < self.height and 0 <= c < self.width and self.walls[r][c] == False:
                neighbours.append((action, (r, c)))
        return neighbours

    def door_at(self, loc):
        return self.door_map.get(loc)

    def key_at(self, loc):
        return self.key_map.get(loc)

    def box_at(self, loc):
        return self.box_map.get(loc)

    def dropzone_for(self, box_label):
        return self.dropzone_map.get(box_label)

    def build_fact_action_index(self):
        index = {}

        for action in self.relaxed_actions:
            for fact in action.preconds:
                index.setdefault(fact, []).append(action)

        self.fact_action_index = index
# defining a class to represent the states in our problem
class State:
    __slots__ = ('soko_at', 'box_carried', 'boxes_at', 'keys_owned', 'keys_at', '_hash')
    
    def __init__(self, soko_at, box_carried, boxes_at, keys_owned, keys_at):
        self.soko_at = soko_at
        self.box_carried = box_carried if box_carried else "-"
        # Store as frozensets of tuples for hashability
        self.boxes_at = frozenset(boxes_at)
        self.keys_owned = frozenset(keys_owned)
        self.keys_at = frozenset(keys_at)
        self._hash = None
    
    def __hash__(self):
        if self._hash is None:
            self._hash = hash((
                self.soko_at,
                self.box_carried,
                self.boxes_at,
                self.keys_owned,
                self.keys_at
            ))
        return self._hash
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return False
        return (
            self.soko_at == other.soko_at and
            self.box_carried == other.box_carried and
            self.boxes_at == other.boxes_at and
            self.keys_owned == other.keys_owned and
            self.keys_at == other.keys_at
        )


def update_state(soko_at, box_carried, boxes_at, keys_owned, keys_at):
    return State(soko_at, box_carried, boxes_at, keys_owned, keys_at)

# defining the possible actions in a state
def transition(state: State, maze: Maze):
    successors = []
    robot = state.soko_at
    box_carried = state.box_carried if state.box_carried != "-" else None
    boxes_positions = state.boxes_at
    keys_owned = state.keys_owned
    keys_positions = state.keys_at

    # Build box lookup for current state
    box_lookup = {(r, c): label for r, c, label in boxes_positions}

    # take_key action
    key_id = maze.key_at(robot)
    if key_id is not None and key_id not in keys_owned:
        new_keys_owned = keys_owned | {key_id}
        new_keys = frozenset(k for k in keys_positions if k[2] != key_id)
        successors.append((
            f"take_key {key_id}", 
            update_state(robot, box_carried, boxes_positions, new_keys_owned, new_keys)
        ))

    # lift_box action
    if box_carried is None:
        box = box_lookup.get(robot)
        if box is not None:
            new_boxes = frozenset((r, c, b) for r, c, b in boxes_positions if (r, c) != robot)
            successors.append((
                f"lift_box {box}", 
                update_state(robot, box, new_boxes, keys_owned, keys_positions)
            ))
    
    # drop_box action
    if box_carried is not None:
        zone_position = maze.dropzone_for(box_carried)
        if zone_position is not None and zone_position == robot:
            successors.append((
                f"drop_box {box_carried}", 
                update_state(robot, None, boxes_positions, keys_owned, keys_positions)
            ))
    
    # move and move_through_door actions
    for direction, neighbour in maze.valid_neighbours(robot):
        door = maze.door_at(neighbour)
        if door is None:
            successors.append((
                f"move_{direction}", 
                update_state(neighbour, box_carried, boxes_positions, keys_owned, keys_positions)
            ))
        else:
            if door in keys_owned:
                successors.append((
                    f"move_{direction}_through_door {door}", 
                    update_state(neighbour, box_carried, boxes_positions, keys_owned, keys_positions)
                ))

    return successors

# defining the goal
def is_goal(state: State):
    return len(state.boxes_at) == 0 and state.box_carried == "-"

#manhattan distance function
def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# euclidean distance function
def euclidean(a, b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)


# implementing the hFF heuristic
#_______________________________________________________________________#

def build_relaxed_actions(maze: Maze):
    actions = []

    # movement actions (ignore walls, doors, keys)
    for r in range(maze.height):
        for c in range(maze.width):
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = r + dr, c + dc
                if 0 <= nr < maze.height and 0 <= nc < maze.width:
                    actions.append(
                        RelaxedAction(
                            f"move({r},{c})->({nr},{nc})",
                            {("robot-at", r, c)},
                            {("robot-at", nr, nc)}
                        )
                    )

    # take key
    for r, c, k in maze.keys:
        actions.append(
            RelaxedAction(
                f"take-key-{k}",
                {("robot-at", r, c), ("key-at", k, r, c)},
                {("key-owned", k)}
            )
        )

    # lift box
    for r, c, b in maze.boxes:
        actions.append(
            RelaxedAction(
                f"lift-box-{b}",
                {("robot-at", r, c), ("box-at", b, r, c)},
                {("carrying", b)}
            )
        )

    # drop box
    for b, (dzr, dzc) in maze.dropzone_map.items():
        actions.append(
            RelaxedAction(
                f"drop-box-{b}",
                {("carrying", b), ("robot-at", dzr, dzc)},
                {("box-at", b, dzr, dzc)}
            )
        )

    return actions


class RelaxedAction:
    def __init__(self, name, preconds, add_effects):
        self.name = name
        self.preconds = frozenset(preconds)
        self.add_effects = frozenset(add_effects)

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return self.name == other.name

# extract the propositional facts of a state for the hFF heuristic
def extract_propositions(state: State, maze: Maze):
    propositions = set()

    # robot position
    r, c = state.soko_at
    propositions.add(("robot-at", r, c))

    # carried box
    if state.box_carried != "-":
        propositions.add(("carrying", state.box_carried))

    # boxes on ground
    for br, bc, label in state.boxes_at:
        propositions.add(("box-at", label, br, bc))

    # owned keys
    for k in state.keys_owned:
        propositions.add(("key-owned", k))

    # keys still in maze
    for kr, kc, label in state.keys_at:
        propositions.add(("key-at", label, kr, kc))

    return propositions

def goal_facts(maze: Maze):
    goals = set()
    for b, (r, c) in maze.dropzone_map.items():
        goals.add(("box-at", b, r, c))
    return goals

def build_relaxed_graph(initial_facts, relaxed_actions, goal_facts, max_layers=300):
    fact_layers = [set(initial_facts)]
    action_layers = []

    for _ in range(max_layers):
        applicable = [
            a for a in relaxed_actions
            if a.preconds.issubset(fact_layers[-1])
        ]

        action_layers.append(applicable)

        new_facts = set(fact_layers[-1])
        for a in applicable:
            new_facts |= a.add_effects

        if new_facts == fact_layers[-1]:
            break

        fact_layers.append(new_facts)

        if goal_facts.issubset(new_facts):
            break

    return fact_layers, action_layers

def extract_relaxed_plan(fact_layers, action_layers, goal_facts):

    if not goal_facts.issubset(fact_layers[-1]):
        return float('inf')
    
    # Memoize which facts are achieved at which level
    fact_achievers = {}
    
    for level, actions in enumerate(action_layers):
        for action in actions:
            for effect in action.add_effects:
                if effect not in fact_achievers:
                    fact_achievers[effect] = (level, action)
    
    # Backward search
    goals = set(goal_facts)
    relaxed_plan = set()
    processed = set()
    
    def extract_recursive(fact, level):
        if fact in fact_layers[0]:
            return
        if fact in processed:
            return
        processed.add(fact)
        
        if fact in fact_achievers:
            lvl, action = fact_achievers[fact]
            if lvl <= level:
                relaxed_plan.add(action)
                # Recursively extract preconditions
                for precond in action.preconds:
                    extract_recursive(precond, lvl)
    
    # Extract for each goal
    for goal in goals:
        extract_recursive(goal, len(action_layers) - 1)
    
    return len(relaxed_plan)

_hff_cache = {}
_cache_limit = 10000


def h_ff(state: State, maze: Maze):
    achieved = set(extract_propositions(state, maze))
    goals = goal_facts(maze)

    if goals.issubset(achieved):
        return 0

    # Track how many preconditions of each action are still missing
    remaining_preconds = {}
    for action in maze.relaxed_actions:
        remaining_preconds[action] = len(action.preconds)

    # Queue of facts that were newly achieved
    fact_queue = list(achieved)
    relaxed_plan_length = 0

    while fact_queue:
        new_queue = []

        for fact in fact_queue:
            for action in maze.fact_action_index.get(fact, []):
                remaining_preconds[action] -= 1

                if remaining_preconds[action] == 0:
                    for eff in action.add_effects:
                        if eff not in achieved:
                            achieved.add(eff)
                            new_queue.append(eff)

        relaxed_plan_length += 1

        if goals.issubset(achieved):
            return relaxed_plan_length

        fact_queue = new_queue

    return float("inf")


#______________________________________________________________________________________________________



# heuristic function. It can be based on manhattan distance or euclidean distance
def heuristic(state: State, maze: Maze, metric='manhattan'):
    if metric == "h_ff": #<- this was added for hff heuristic
        return h_ff(state, maze)
    
    boxes_positions = state.boxes_at
    carried = None if state.box_carried == "-" else state.box_carried
    robot = state.soko_at
    keys_owned = state.keys_owned
    keys_remaining = state.keys_at

    # if there are no boxes on the grid and if there are no boxes carried (i.e. if the goal is reached) the heuristic is 0
    if not boxes_positions and carried is None:
        return 0

    # initialise which distance function is used
    distance = manhattan if metric == 'manhattan' else euclidean

    # initialise the heuristic value h
    h = 0

    # add the h value for all the keys and doors 
    doors_needed = set()
    for door_pos, door_label in maze.door_map.items():
        if door_label not in keys_owned:
            doors_needed.add(door_label)

    for key_r, key_c, key_label in keys_remaining:
        if key_label in doors_needed:
            h += distance(robot, (key_r, key_c))

    # if a box is carried, we add only the distance to the correct dropzone to the h value
    if carried is not None:
        dz = maze.dropzone_for(carried)
        if dz is not None:
            h += distance(robot, dz)
        
        # Add minimum distance for remaining boxes
        for r, c, b in boxes_positions:
            dz = maze.dropzone_for(b)
            if dz is not None:
                h += distance((r, c), dz)
    else:
        # If no box is carried, find nearest box + its delivery cost
        if boxes_positions:
            min_cost = float('inf')
            for r, c, b in boxes_positions:
                pos = (r, c)
                dz = maze.dropzone_for(b)
                if dz is not None:
                    cost = distance(robot, pos) + distance(pos, dz)#<- to add the delivery cost or not to add delivery cost
                    min_cost = min(min_cost, cost)
            
            if min_cost != float('inf'):
                h += min_cost
            
            # Add remaining boxes' distances
            for r, c, b in boxes_positions:
                pos = (r, c)
                dz = maze.dropzone_for(b)
                if dz is not None:
                    h += distance(pos, dz)
        print(int(h))
    return int(h)

# breadth-first search algorithm
def bfs(maze: Maze, time_limit=30.0):

    start = update_state(maze.soko, None, 
                        frozenset(maze.boxes), 
                        frozenset(), 
                        frozenset(maze.keys))
    start_time = time.time()
    
    q = deque([start])
    parent = {start: (None, None)}
    explored = {start}
    expanded = 0
    generated = 0

    while q:
        if time.time() - start_time > time_limit:
            return None, None, None, {"reason": "timeout", "expanded": expanded, "generated": generated}
        
        state = q.popleft()
        generated += 1
        
        if is_goal(state):
            actions = []
            solution_states = [state]
            cur = state
            while parent[cur][0] is not None:
                pstate, action = parent[cur]
                actions.append(action)
                solution_states.append(pstate)
                cur = pstate
            actions.reverse()
            solution_states.reverse()
            return actions, solution_states, None, {"expanded": expanded, "generated": generated, "time": time.time() - start_time}
        
        expanded += 1
            
        for action, succ in transition(state, maze):
            if succ not in explored:
                explored.add(succ)
                parent[succ] = (state, action)
                q.append(succ)

    return None, None, None, {"reason": "exhausted", "expanded": expanded, "generated": generated}

# A* algorithm
def astar(maze: Maze, time_limit=30.0, metric='manhattan'):
    start = update_state(maze.soko, None, 
                        frozenset(maze.boxes), 
                        frozenset(), 
                        frozenset(maze.keys))
    start_time = time.time()
    
    frontier = []
    counter = 0
    g_scores = {start: 0}
    parent = {start: (None, None)}
    heappush(frontier, (heuristic(start, maze, metric), 0, counter, start))
    
    expanded = 0
    generated = 0
    closed = set()

    while frontier:
        if time.time() - start_time > time_limit:
            return None, None, None, {"reason": "timeout", "expanded": expanded, "generated": generated}

        f, g, _, state = heappop(frontier)
        generated += 1
        
        if state in closed:
            continue
        closed.add(state)
        
        if is_goal(state):
            actions = []
            solution_states = [state]
            cur = state
            while parent[cur][0] is not None:
                pstate, action = parent[cur]
                actions.append(action)
                solution_states.append(pstate)
                cur = pstate
            actions.reverse()
            solution_states.reverse()
            return actions, solution_states, g_scores[state], {
                "expanded": expanded, 
                "generated": generated, 
                "time": time.time() - start_time
            }
        
        expanded += 1
        
        cur_g = g_scores[state]
        for action, succ in transition(state, maze):
            if succ in closed:
                continue
                
            new_g = cur_g + 1
            if succ not in g_scores or new_g < g_scores[succ]:
                g_scores[succ] = new_g
                parent[succ] = (state, action)
                counter += 1
                fscore = new_g + heuristic(succ, maze, metric)
                heappush(frontier, (fscore, new_g, counter, succ))

    return None, None, None, {"reason": "exhausted", "expanded": expanded, "generated": generated}

# Greedy best-first search algorithm
def greedy_bfs(maze:Maze, time_limit=30.0, metric='manhattan'):
    start = update_state(maze.soko, None, 
                        frozenset(maze.boxes), 
                        frozenset(), 
                        frozenset(maze.keys))
    start_time = time.time()
    frontier = []
    expanded_positions = []
    counter = 0

    parent = {start: (None, None)}
    heappush(frontier, (heuristic(start, maze, metric), counter, start))
    heappush(expanded_positions, start.soko_at)
    
    expanded = 0
    generated = 0
    closed = set()

    while frontier:
        if time.time() - start_time > time_limit:
            return None, None, None, {"reason": "timeout", "expanded": expanded, "generated": generated}

        h, _, state = heappop(frontier)
        generated += 1
        
        if state in closed:
            continue
        closed.add(state)
        heappush(expanded_positions, state.soko_at)
        
        if is_goal(state):
            actions = []
            solution_states = [state]
            cur = state
            while parent[cur][0] is not None:
                pstate, action = parent[cur]
                actions.append(action)
                solution_states.append(pstate)
                cur = pstate
            actions.reverse()
            solution_states.reverse()
            return actions, solution_states, len(actions), {
                "expanded": expanded, 
                "generated": generated, 
                "time": time.time() - start_time
            }
        
        expanded += 1
        
        for action, succ in transition(state, maze):
            if succ in closed:
                continue
            
            if succ not in parent:
                parent[succ] = (state, action)
                counter += 1
                h_score = heuristic(succ, maze, metric)
                heappush(frontier, (h_score, counter, succ))

    return None, None, None, {"reason": "exhausted", "expanded": expanded, "generated": generated}

#enforced hill climbing algorithm
def enforced_hc(maze:Maze, time_limit=30.0, metric="manhattan"):

    start = update_state(maze.soko, None, 
                        frozenset(maze.boxes), 
                        frozenset(), 
                        frozenset(maze.keys))
    start_time = time.time()
    q = deque([start])
    explored = {start}
    parent = {start: (None, None)}
    expanded = 0
    generated = 0
    best_h = heuristic(start, maze, metric)

    while q:
        if time.time() - start_time > time_limit:
            return None, None, None, {"reason": "timeout", "expanded": expanded, "generated": generated}

        state = q.popleft()
        generated += 1

        if is_goal(state):
            actions = []
            solution_states = [state]
            cur = state
            while parent[cur][0] is not None:
                pstate, action = parent[cur]
                actions.append(action)
                solution_states.append(pstate)
                cur = pstate
            actions.reverse()
            solution_states.reverse()
            return actions, solution_states, None, {"expanded": expanded, "generated": generated, "time": time.time() - start_time}
        expanded += 1

        for action, succ in transition(state, maze):
            if succ not in explored:
                explored.add(succ)
                parent[succ] = (state, action)
                new_h = heuristic(succ, maze, metric)
                q.append(succ)

                if new_h < best_h:
                    best_h = new_h
                    q = deque([succ])
                    break

    return None, None, None, {"reason" : "exhausted", "expanded": expanded, "generated": generated}


#__________________________________________________
def initial_state(maze: Maze) -> State:
    return State(
        soko_at=maze.soko,
        box_carried="-",
        boxes_at=frozenset(maze.boxes),
        keys_owned=frozenset(),
        keys_at=frozenset(maze.keys)
    )

def apply_action(state: State, action: str, maze: Maze) -> State:
    """
    Applies a single action to a state.
    Raises ValueError if the action is not applicable.
    """
    successors = transition(state, maze)

    for act, succ in successors:
        if act == action:
            return succ

    raise ValueError(f"Action '{action}' is not applicable in state {state}")

def validate_plan(plan, maze: Maze):
    """
    Validates a plan by executing it from the initial state.
    Returns (True, None) if valid.
    Returns (False, error_message) otherwise.
    """
    state = initial_state(maze)

    for step, action in enumerate(plan, start=1):
        try:
            state = apply_action(state, action, maze)
        except ValueError as e:
            return False, f"Step {step}: {e}"

    if not is_goal(state):
        return False, "Final state does not satisfy the goal condition"

    return True, None

#_______________________________________________________________________________________#
#________________________________GUI and Visualisation _________________________________#
#_______________________________________________________________________________________#


COLORS = {
    "wall": "black",
    "empty": "white",
    "robot": "blue",
    "box": "orange",
    "drop": "green",
    "key": "gold",
    "door_locked": "red",
    "door_open": "pink",
    "expanded": "#8E8E8E",
    "path": "#90EE90"
}

class SokoGUI:
    
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Soko Automated Planner")
        self.root.geometry("1280x700")
        
        self.maze = None
        self.visualiser = None
        self.solution = None
        self.actions = None
        self.animating = False
        
        self.setup_ui()
    
    def setup_ui(self):
        # Top control panel
        control_frame = tk.Frame(self.root, bg="#f0f0f0", pady=10)
        control_frame.pack(side=tk.TOP, fill=tk.X)
        
        # File selection
        tk.Button(control_frame, text="Load Maze", 
                 command=self.load_maze, 
                 bg="#4CAF50", fg="white", 
                 font=("Arial", 10, "bold")).pack(side=tk.LEFT, padx=5)
        
        self.maze_label = tk.Label(control_frame, text="No maze loaded", 
                                   font=("Arial", 9))
        self.maze_label.pack(side=tk.LEFT, padx=10)
        
        # Algorithm selection
        tk.Label(control_frame, text="Algorithm:", 
                font=("Arial", 9)).pack(side=tk.LEFT, padx=(30,5))
        
        self.algorithm_var = tk.StringVar(value="astar")
        algorithms = [
            ("Breadth-First Search", "bfs"),
            ("A*", "astar"),
            ("Greedy Best-First", "greedy"),
            ("Enforced Hill Climb", "ehc")
        ]        
        for text, value in algorithms:
            tk.Radiobutton(control_frame, text=text, 
                          variable=self.algorithm_var, 
                          value=value,
                          font=("Arial", 9)).pack(side=tk.LEFT)
            
        
        # Heuristic selection
        tk.Label(control_frame, text="Heuristic:", 
                font=("Arial", 9)).pack(side=tk.LEFT, padx=(30,5))
        
        self.heuristic_var = tk.StringVar(value="manhattan")
        tk.Radiobutton(control_frame, text="Manhattan", 
                      variable=self.heuristic_var, 
                      value="manhattan",
                      font=("Arial", 9)).pack(side=tk.LEFT)
        tk.Radiobutton(control_frame, text="Euclidean", 
                variable=self.heuristic_var, 
                value="euclidean",
                font=("Arial", 9)).pack(side=tk.LEFT)
        
        tk.Radiobutton(
                        control_frame,
                        text="H-ff (relaxed plan)",
                        variable=self.heuristic_var,
                        value="h_ff",
                        font=("Arial", 9)
                    ).pack(side=tk.LEFT)

        # Solve button
        tk.Button(control_frame, text="Solve", 
                 command=self.solve_maze,
                 bg="#2196F3", fg="white",
                 font=("Arial", 10, "bold")).pack(side=tk.LEFT, padx=(20,5))
        
        # Main content area
        content_frame = tk.Frame(self.root)
        content_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # canvas for maze
        canvas_frame = tk.LabelFrame(content_frame, text="Maze Visualization", 
                                     font=("Arial", 10, "bold"))
        canvas_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        self.canvas = tk.Canvas(canvas_frame, bg="white", 
                               width=700, height=280)
        self.canvas.pack(padx=5, pady=5)
        
        # info panels
        info_frame = tk.LabelFrame(content_frame, text="Information", 
                                   font=("Arial", 10, "bold"),
                                   width=300)
        info_frame.pack(side=tk.RIGHT, fill=tk.BOTH, padx=(10,0))
        info_frame.pack_propagate(False)
        
        # Stats display
        self.stats_text = tk.Text(info_frame, height=15, width=35, 
                                 font=("Courier", 9),
                                 state=tk.DISABLED)
        self.stats_text.pack(padx=5, pady=5, fill=tk.BOTH, expand=True)
        
        # Solution display
        solution_label = tk.Label(info_frame, text="Solution Path:", 
                                 font=("Arial", 9, "bold"))
        solution_label.pack(pady=(10,5))
        
        solution_frame = tk.Frame(info_frame)
        solution_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        scrollbar = tk.Scrollbar(solution_frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.solution_text = tk.Text(solution_frame, height=15, 
                                     font=("Courier", 8),
                                     yscrollcommand=scrollbar.set,
                                     state=tk.DISABLED)
        self.solution_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.config(command=self.solution_text.yview)
        
        # Animation controls
        anim_frame = tk.Frame(info_frame)
        anim_frame.pack(pady=10)
        
        tk.Button(anim_frame, text="▶ Animate", 
                 command=self.animate_solution,
                 bg="#FF9800", fg="white").pack(side=tk.LEFT, padx=2)
        
        tk.Label(anim_frame, text="Speed:", 
            font=("Arial", 9)).pack(side=tk.LEFT, padx=(20,5))
        
        self.speed_var = tk.IntVar(value=20)
        tk.Scale(anim_frame, from_=1, to=100, 
                variable=self.speed_var,
                orient=tk.HORIZONTAL, 
                length=75).pack(side=tk.LEFT)
    
    # load and handle the maze .txt file
    def load_maze(self):
        filename = filedialog.askopenfilename(
            title="Select Maze File",
            filetypes=[("Text files", "*.txt"), ("All files", "*.*")]
        )
        if filename:
            try:
                self.maze = Maze(filename)
                self.maze.relaxed_actions = build_relaxed_actions(self.maze) #<- This was added last for the hFF heuristic
                self.maze.build_fact_action_index() #<- this too
                self.maze_label.config(text=f"Loaded: {filename.split('/')[-1]}")
                self.visualiser = SokoVisualiser(self.maze, self.canvas)
                self.visualiser.draw_maze()
                self.update_stats("Maze loaded successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load maze:\n{e}")
    
    # solving the maze using the specified algorithm
    def solve_maze(self):
        if self.maze is None:
            messagebox.showwarning("Warning", "Please load a maze first!")
            return
        
        algorithm = self.algorithm_var.get()
        heuristic = self.heuristic_var.get()
        
        self.update_stats(f"Running {algorithm.upper()}...")
        self.root.update()
        
        solvers = {
            "astar": lambda m: astar(m, metric=heuristic, time_limit=60.0),
            "bfs": lambda m: bfs(m, time_limit=60.0),
            "greedy": lambda m: greedy_bfs(m, metric=heuristic, time_limit=60.0),
            "ehc": lambda m: enforced_hc(m, metric=heuristic, time_limit=60.0)
        }
        
        try:
            actions, plan, cost, stats = solvers[algorithm](self.maze)

            if plan is None:
                self.update_stats(f"No solution found!\nReason: {stats.get('reason', 'unknown')}\n" +
                                f"States expanded: {stats['expanded']}\n" +
                                f"States generated: {stats['generated']}")
                messagebox.showinfo("Result", "No solution found!")
            else:
                self.solution = plan
                self.actions = actions
                self.display_solution(actions, cost, stats)
                messagebox.showinfo("Success", f"Solution found!\nSteps: {len(actions)}")

        except Exception as e:
            messagebox.showerror("Error", f"Solver failed:\n{e}")

        # validation step
        valid, error = validate_plan(actions, self.maze)

        if not valid:
            messagebox.showerror(
            "Invalid Plan",
            f"The planner returned an invalid plan:\n{error}"
            )
            self.update_stats("Planner returned an invalid plan.")
            messagebox.showerror("Invalid Plan", error)
            return
        #___________________

    def display_solution(self, actions, cost, stats):
        # Update stats
        stats_text = f"""
Algorithm: {self.algorithm_var.get().upper()}
Heuristic: {self.heuristic_var.get()}
━━━━━━━━━━━━━━━━━━━━━━━━
Solution Length: {len(actions)} steps
Path Cost: {cost}
States Expanded: {stats['expanded']}
States generated: {stats['generated']}
Time Taken: {stats.get('time', 0):.3f}s
━━━━━━━━━━━━━━━━━━━━━━━━
Status: Solution found ✓
        """
        
        self.stats_text.config(state=tk.NORMAL)
        self.stats_text.delete(1.0, tk.END)
        self.stats_text.insert(1.0, stats_text)
        self.stats_text.config(state=tk.DISABLED)
        
        # Display solution steps
        self.solution_text.config(state=tk.NORMAL)
        self.solution_text.delete(1.0, tk.END)
        for i, action in enumerate(actions, 1):
            self.solution_text.insert(tk.END, f"{i:3d}. {action}\n")
        self.solution_text.config(state=tk.DISABLED)

    def update_stats(self, message):
        self.stats_text.config(state=tk.NORMAL)
        self.stats_text.delete(1.0, tk.END)
        self.stats_text.insert(1.0, message)
        self.stats_text.config(state=tk.DISABLED)

    def update_solution(self, message):
        self.solution_text.config(state=tk.NORMAL)
        self.solution_text.delete(1.0, tk.END)
        self.solution_text.insert(1.0, message)
        self.solution_text.config(state=tk.DISABLED)

    def animate_search(self, expanded):
        if self.solution is None:
            messagebox.showwarning("Warning", "No process to animate!")
            return
        
        if self.animating:
            return
        
        self.animating = True
        self.visualiser.draw_maze()
        
        states = self.solution.copy()

        if not states:
            return
        
        def step():
            state = states.pop(0)
            self.visualiser.render_state(state)
            time.sleep(1.0 / self.speed_var.get())

        for i, action in enumerate(self.actions):
            if not self.animating:
                break
            
            self.update_solution(f"Step {i+1}/{len(self.actions)}:\n{action}")
            step()
            self.root.update()
            time.sleep(1.0 / self.speed_var.get())
    
        self.animating = False

    
    def animate_solution(self):
        if self.solution is None:
            messagebox.showwarning("Warning", "No solution to animate!")
            return
        
        if self.animating:
            return
            
        self.animating = True
        self.visualiser.draw_maze()
        
        states = self.solution.copy()

        if not states:
            return
        
        def step():
            state = states.pop(0)
            self.visualiser.render_state(state)
            time.sleep(1.0 / self.speed_var.get())

        for i, action in enumerate(self.actions):
            if not self.animating:
                break
            
            self.update_solution(f"Step {i+1}/{len(self.actions)}:\n{action}")
            step()
            self.root.update()
            time.sleep(1.0 / self.speed_var.get())
    
        self.animating = False
    
    def run(self):
        self.root.mainloop()


class SokoVisualiser:
    
    def __init__(self, maze, canvas):
        self.maze = maze
        self.canvas = canvas
        self.last_robot_pos = None
        self.rects = {}
        self.cell_size = min(
            canvas.winfo_reqwidth() // maze.width,
            canvas.winfo_reqheight() // maze.height
        )
    
    def draw_maze(self):
        self.canvas.delete("all")
        self.rects = {}
        
        for r in range(self.maze.height):
            for c in range(self.maze.width):
                x1 = c * self.cell_size
                y1 = r * self.cell_size
                x2 = x1 + self.cell_size
                y2 = y1 + self.cell_size
                
                color = COLORS["wall"] if self.maze.walls[r][c] else COLORS["empty"]
                rect = self.canvas.create_rectangle(x1, y1, x2, y2, 
                                                   fill=color, outline="gray")
                self.rects[(r, c)] = rect

        # Draw special cells
        for x, y, _ in self.maze.dropzones:
            self.canvas.itemconfig(self.rects[(x, y)], fill=COLORS["drop"])
        
        for x, y, _ in self.maze.keys:
            self.canvas.itemconfig(self.rects[(x, y)], fill=COLORS["key"])
        
        for x, y, _ in self.maze.doors:
            self.canvas.itemconfig(self.rects[(x, y)], fill=COLORS["door_locked"])
        
        for x, y, _ in self.maze.boxes:
            self.canvas.itemconfig(self.rects[(x, y)], fill=COLORS["box"])
        
        # Draw robot
        self.canvas.itemconfig(self.rects[self.maze.soko], fill=COLORS["robot"])
    
    def render_state(self, state, expanded=[]):

            # Ensuring that we only draw the current robot position and not all the positions that the robot occupied
        if self.last_robot_pos is not None:
            r, c = self.last_robot_pos
            if (r, c) in self.maze.walls:
                color = COLORS["wall"]
            elif (r, c) in self.maze.dropzone_map.values():
                color = COLORS["drop"]
            else:
                color = COLORS["empty"]
            self.canvas.itemconfig(self.rects[(r, c)], fill=color)

            # Draw current state
        for x, y, _ in state.boxes_at:
            self.canvas.itemconfig(self.rects[(x, y)], fill=COLORS["box"])
        
        self.canvas.itemconfig(self.rects[state.soko_at], fill=COLORS["robot"])
        self.last_robot_pos = state.soko_at
        self.canvas.update()

# Main execution
if __name__ == "__main__":
    app = SokoGUI()
    app.run()